In [1]:
import firebase_admin
from pathlib import Path
from firebase_admin import credentials
from firebase_admin import firestore
import warnings
import os
warnings.filterwarnings('ignore')

In [2]:
# Use the path to your service account key file
PARENT_DIR = Path('__file__').resolve().parent
print(PARENT_DIR)


C:\Users\encaa\OneDrive\Desktop\Mac_Geo\Geo_location\xcor


In [3]:
cred = credentials.Certificate(os.path.join(PARENT_DIR,"serviceAccountKey.json"))

# Initialize the app with your project's credentials
# You can also specify the database URL if it's not the default
firebase_admin.initialize_app(cred, {
    'projectId': 'geolocation-app-b549e',
})

# Now you can use the Firebase services
# For example, to access Firestore:
db = firestore.client()

In [4]:
def get_all_sessions():
    """Retrieve all session data from Firestore"""
    if not db:
        print("Firebase not initialized. Please check your configuration.")
        return None
    
    try:
        # Get all documents from the 'locations' collection
        locations_ref = db.collection('sessions')
        docs = locations_ref.stream()
        
        locations_data = []
        for doc in docs:
            data = doc.to_dict()
            data['document_id'] = doc.id  # Add document ID for reference
            locations_data.append(data)
        
        print(f"Retrieved {len(locations_data)} location records")
        return locations_data
    except Exception as e:
        print(f"Error retrieving data: {e}")
        return None

In [5]:
sessions_data = get_all_sessions()

if sessions_data:
    print(f"\nSuccessfully connected! Found {len(sessions_data)}  records.")
    print("\nSample data structure:")
    if sessions_data:
        sample = sessions_data[1]
        for key, value in sample.items():
            print(f"  {key}: {value}")
else:
    print("Failed to retrieve data. Please check your Firebase configuration.")


Retrieved 4 location records

Successfully connected! Found 4  records.

Sample data structure:
  startTime: 2025-09-30T19:42:00.000Z
  participants: [{'role': 'created', 'username': 'o2', 'audioUrl': 'https://firebasestorage.googleapis.com/v0/b/geolocation-app-b549e.firebasestorage.app/o/audio%2Foffice2_o2_2025-09-30_14%3A42.webm?alt=media&token=c06f79b5-8b3a-49fe-8928-1c15f54b57a2', 'location': {'accuracy': 4027.2869876813356, 'longitude': -90.4744285, 'latitude': 30.5155244, 'timestamp': '30/09/2025, 14:41:35'}, 'joinedAt': '2025-09-30T19:41:34.722Z', 'deviceName': 'Windows PC (Chrome)'}]
  events: [{'type': 'created', 'by': 'o2', 'timestamp': '2025-09-30T19:41:34.722Z'}, {'type': 'saved', 'by': 'o2', 'timestamp': '2025-09-30T19:42:20.523Z'}]
  document_id: office2


In [9]:
sessions_data[2]

{'startTime': '2025-09-28T16:52:00.000Z',
 'participants': [{'role': 'created',
   'username': 'a1',
   'location': {'accuracy': 1070.9677895959417,
    'longitude': -90.4757248,
    'latitude': 30.5135616,
    'timestamp': '28/09/2025, 11:51:38'},
   'audioUrl': 'https://firebasestorage.googleapis.com/v0/b/geolocation-app-b549e.firebasestorage.app/o/audio%2Ft1_a1_2025-09-28_11%3A52.wav?alt=media&token=326792e3-0a30-412a-be7e-2e8b837a9956',
   'joinedAt': '2025-09-28T16:51:38.410Z',
   'deviceName': 'Windows PC (Chrome)'},
  {'joinedAt': '2025-09-28T16:51:53.151Z',
   'username': 'a2',
   'audioUrl': 'https://firebasestorage.googleapis.com/v0/b/geolocation-app-b549e.firebasestorage.app/o/audio%2Ft1_a2_2025-09-28_11%3A52.wav?alt=media&token=7a1df1bf-0ae8-4adc-af75-96c56b6a9243',
   'location': {'accuracy': 1070.9677895959417,
    'longitude': -90.4757248,
    'latitude': 30.5135616,
    'timestamp': '28/09/2025, 11:51:54'},
   'role': 'joined',
   'deviceName': 'Windows PC (Chrome)'}],


In [10]:
sessions_data[0]['participants'][0]['audioUrl']

'https://firebasestorage.googleapis.com/v0/b/geolocation-app-b549e.firebasestorage.app/o/audio%2Fchurch_a2_2025-09-25_19%3A00.webm?alt=media&token=19f9cdd6-3549-4469-a930-71b09f923e4a'

In [7]:
import librosa
import io

import os
from pathlib import Path

import librosa
import numpy as np
import matplotlib.pyplot as plt

from scipy.io import wavfile
from scipy.fft import fft, fftfreq

In [ ]:
def plot_audio_analysis(audio_path):
    # Load audio file
    print('INsert')
    y, sr = librosa.load(audio_path, sr=None)  # y = waveform, sr = sample rate
    print('Read data')
    duration = len(y) / sr
    time = np.linspace(0, duration, len(y))

    # FFT
    N = len(y)
    yf = fft(y)
    xf = fftfreq(N, 1 / sr)[:N // 2]
    spectrum = np.abs(yf[:N // 2])

    # Spectrogram
    S = librosa.stft(y)
    S_db = librosa.amplitude_to_db(np.abs(S), ref=np.max)

    # Plot
    fig, axs = plt.subplots(3, 1, figsize=(12, 10))

    # Waveform
    axs[0].plot(time, y, color='blue')
    axs[0].set_title('Waveform')
    axs[0].set_xlabel('Time [s]')
    axs[0].set_ylabel('Amplitude')

    # FFT Spectrum
    axs[1].plot(xf, spectrum, color='green')
    axs[1].set_title('FFT Spectrum')
    axs[1].set_xlabel('Frequency [Hz]')
    axs[1].set_ylabel('Magnitude')
    axs[1].set_xlim(0, sr / 2)  # Limit to Nyquist

    # Spectrogram
    img = librosa.display.specshow(S_db, sr=sr, x_axis='time', y_axis='log', ax=axs[2], cmap='gray_r')
    axs[2].set_title('Spectrogram (dB)')
    #fig.colorbar(img, ax=axs[2], format='%+2.0f dB')

    plt.tight_layout()
    plt.show()

In [10]:
import requests
par_session = sessions_data[1]['participants'] 
for i in range(len(par_session)):
    try:
        audio = par_session[i]['audioUrl']
        response = requests.get(audio)
        audio_name = f'audio_{par_session[i]["username"]}.wav'
        with open(audio_name, 'wb') as f:
            f.write(response.content)

        #plot_audio_analysis(audio_name)
        
    except Exception as e:
        print(f"Error downloading audio for {par_session[i]['username']}: {e}")


# 